# Jakub Rymarski 

### Raport - Praca domowa nr 4
Praca polegała na rozwiązaniu 5 zadań dwoma sposobami. Jednym sposobem było wywołanie polecenia pandas.read_sql_query("""zapytanie SQL"""), a drugi miał wykorzystywać "zwykłe metody i funkcje z pakietu pandas.
W raporcie przedstawiam te sposoby roziwązania i porównuję czy rzeczywiście dają ten sam wynik (przy użyciu metody ".equals()").

In [5]:
import pandas as pd
import numpy as np
import os, os.path
import sqlite3
import tempfile


# Importowanie danych

Tags = pd.read_csv("Tags.csv.gz", compression = 'gzip')
Badges = pd.read_csv("Badges.csv.gz", compression = 'gzip')
Comments = pd.read_csv("Comments.csv.gz", compression = 'gzip')
PostLinks = pd.read_csv("PostLinks.csv.gz", compression = 'gzip')
Posts = pd.read_csv("Posts.csv.gz", compression = 'gzip')
Users = pd.read_csv("Users.csv.gz", compression = 'gzip')
Votes = pd.read_csv("Votes.csv.gz", compression = 'gzip')


baza = os.path.join(tempfile.mkdtemp(), 'przyklad.db')
if os.path.isfile(baza):
    os.remove(baza)
conn = sqlite3.connect(baza)
Tags.to_sql("Tags", conn)
Badges.to_sql("Badges", conn)
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)


# Zadanie 1

# Z wykorzystaniem pandas.read_sql_query

df_sql_1 = pd.read_sql_query("""
SELECT Count, TagName
FROM Tags
WHERE Count > 1000
ORDER BY Count DESC
""", conn)

# Z wykorzystaniem "zwykłych" metod i funkcji

df1 = Tags[Tags["Count"] > 1000]
df1 = df1.sort_values(by = "Count", ascending=False)
df1 = df1.reset_index()
df1 = df1.filter(items=["Count", "TagName"])



# Zadanie 2

# Z wykorzystaniem pandas.read_sql_query

df_sql_2 = pd.read_sql_query("""
SELECT Location, COUNT(*) AS Count
FROM (
SELECT Posts.OwnerUserId, Users.Id, Users.Location
FROM Users
JOIN Posts ON Users.Id = Posts.OwnerUserId
)
WHERE Location NOT IN ('')
GROUP BY Location
ORDER BY Count DESC
LIMIT 10
""", conn)

# Z wykorzystaniem "zwykłych" metod i funkcji

df2 = Users.merge(Posts, left_on="Id", right_on="OwnerUserId", how="inner")
df2 = df2.filter(items=["OwnerUserId", "Id", "Location"])
df2 = df2[df2["Location"]!=""]
df2=df2.Location.value_counts().reset_index().rename(columns = {"index": "Location",
"Location": "Count"})
df2 = df2.sort_values(by = "Count", ascending=False)
df2 = df2.head(10)



# Zadanie 3

# Z wykorzystaniem pandas.read_sql_query

df_sql_3 = pd.read_sql_query("""
SELECT Year, SUM(Number) AS TotalNumber
FROM (
SELECT
Name,
COUNT(*) AS Number,
STRFTIME('%Y', Badges.Date) AS Year
FROM Badges
WHERE Class = 1
GROUP BY Name, Year
)
GROUP BY Year
ORDER BY TotalNumber
""", conn)

# Z wykorzystaniem "zwykłych" metod i funkcji

Badges2=Badges
Badges2["Date"]=Badges2["Date"].str.slice(0, 4)
df3 = Badges2[Badges2["Class"] == 1]
df3=df3.rename(columns = {"Date": "Year"})
df3 = df3.filter(items=["Name", "Year"])
df3=df3.groupby(["Name", "Year"]).size().reset_index(name="Number")
df3=df3.groupby(["Year"]).sum().rename(columns = {"Number": "TotalNumber"})
df3 = df3.sort_values(by = "TotalNumber")
df3=df3.reset_index().rename(columns = {"TotalNumber": "TotalNumber"})



# Zadanie 4

# Z wykorzystaniem pandas.read_sql_query

df_sql_4 = pd.read_sql_query("""
SELECT
Users.AccountId,
Users.DisplayName,
Users.Location,
AVG(PostAuth.AnswersCount) as AverageAnswersCount
FROM
(
SELECT
AnsCount.AnswersCount,
Posts.Id,
Posts.OwnerUserId
FROM (
SELECT Posts.ParentId, COUNT(*) AS AnswersCount
FROM Posts
WHERE Posts.PostTypeId = 2
GROUP BY Posts.ParentId
) AS AnsCount
JOIN Posts ON Posts.Id = AnsCount.ParentId
) AS PostAuth
JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
GROUP BY OwnerUserId
ORDER BY AverageAnswersCount DESC, AccountId ASC
LIMIT 10
""", conn)

# Z wykorzystaniem "zwykłych" metod i funkcji

AnsCount=Posts[Posts["PostTypeId"]==2]
AnsCount=AnsCount.filter(items=["ParentId"])
AnsCount=AnsCount.ParentId.value_counts().reset_index().rename(columns = {"index": "ParentId", "ParentId": "AnswersCount"})

PostAuth= AnsCount.merge(Posts, left_on="ParentId", right_on="Id", how="inner")
PostAuth=PostAuth.filter(items=["AnswersCount", "Id", "OwnerUserId"])
PostAuth = PostAuth.groupby(["OwnerUserId"]).mean().rename(columns = {"AnswersCount": "AverageAnswersCount"})
df4=PostAuth.merge(Users, left_on="OwnerUserId", right_on="AccountId", how="inner")
df4=df4.filter(items=["AccountId", "DisplayName", "Location", "AverageAnswersCount"])
df4= df4.sort_values(by = "AverageAnswersCount", ascending=False).head(10)
df4=df4.sort_values(by="AccountId")
df4= df4.sort_values(by = "AverageAnswersCount", ascending=False)
df4 = df4.where(pd.notnull(df4), None)
df4 = df4.reset_index(drop = True)



# Zadanie 5

# Z wykorzystaniem pandas.read_sql_query

df_sql_5 = pd.read_sql_query("""
SELECT Posts.Title, Posts.Id,
STRFTIME('%Y-%m-%d', Posts.CreationDate) AS Date,
VotesByAge.Votes
FROM Posts
JOIN (
SELECT
PostId,
MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
SUM(Total) AS Votes
FROM (
SELECT
PostId,
CASE STRFTIME('%Y', CreationDate)
WHEN '2021' THEN 'new'
WHEN '2020' THEN 'new'
ELSE 'old'
END VoteDate,
COUNT(*) AS Total
FROM Votes
WHERE VoteTypeId IN (1, 2, 5)
GROUP BY PostId, VoteDate
) AS VotesDates
GROUP BY VotesDates.PostId
HAVING NewVotes > OldVotes
) AS VotesByAge ON Posts.Id = VotesByAge.PostId
WHERE Title NOT IN ('')
ORDER BY Votes DESC
LIMIT 10
""", conn)

# Z wykorzystaniem "zwykłych" metod i funkcji

Votes2=Votes[Votes["VoteTypeId"].isin([1, 2, 5])]
Votes2=Votes2.filter(items=["PostId", "CreationDate"])
Votes2["CreationDate"]=Votes2["CreationDate"].str.slice(0, 4)
Votes2["VoteDate"]=Votes2.CreationDate.apply(lambda x: "new" if x=="2021" or x=="2020" else "old")
VotesDates=Votes2.filter(items=["PostId", "VoteDate"]).groupby(["PostId", "VoteDate"]).size().reset_index()\
    .rename(columns={0: "Total"})

#Z pomocą funkcji nv i ov tworzymy kolumny NewVotes i OldVotes

def nv(x, dat):
    if dat=="new":
        return x
    else:
        return 0

def ov(x, dat):
    if dat=="old":
        return x
    else:
        return 0

VotesDates["New"]=np.vectorize(nv)(VotesDates["Total"], VotesDates["VoteDate"])
VotesDates["Old"]=np.vectorize(ov)(VotesDates["Total"], VotesDates["VoteDate"])
VotesDates=VotesDates.filter(items=["PostId", "New", "Old", "Total"])
VotesDates=VotesDates.groupby("PostId").agg(NewVotes=("New", max),
                                 OldVotes=("Old", max),
                                 Votes=("Total", sum))\
    .reset_index()

VotesByAge=VotesDates[VotesDates.NewVotes>VotesDates.OldVotes]
Posts2=Posts.filter(items=["Title", "Id", "CreationDate"])
Posts2["Date"]=Posts2["CreationDate"].str.slice(0, 10)
Posts2=Posts2[Posts2.Title.notnull()]
Posts2=Posts2.filter(items=["Title", "Id", "Date"])

VotesByAge2=VotesByAge.filter(items=["PostId", "Votes"])
df5=Posts2.merge(VotesByAge2, left_on="Id", right_on="PostId", how="inner")
df5=df5.filter(items=["Title", "Id", "Date", "Votes"])
df5=df5.sort_values(["Votes"], ascending=False).reset_index(drop = True).head(10)

# Nasze wyniki różnią się kolejnością alfabetyczną Tytułu, dlatego możemy posortować również względem Title

df5=df5.sort_values(["Votes", "Title"], ascending=[False, True]).reset_index(drop=True)

conn.close()

### Sprawdzamy czy wyniki są takie same.

In [6]:
print(df_sql_1.equals(df1))
print(df_sql_2.equals(df2))
print(df_sql_3.equals(df3))
print(df_sql_4.equals(df4))
print(df_sql_5.equals(df5))

True
True
True
True
True


Otrzymaliśmy 5 wyników "True", czyli nasze dwa sposoby rozwiązania dają ten sam wynik.

# Podsumowanie
Oczywiście łatwiejszym sposobem rozwiązania jest skorzystanie z gotowej funkcji pandas.read_sql_query("""Zapytanie SQL"""). Drugi sposób był czasochłonny, ale ostatecznie nie aż tak trudny, ponieważ w pakiecie pandas możemy korzystać z wielu pomocnych funkcji, które ułatwiają nam zadanie.
Warto też porównać ten projekt z pracą domową nr 2, w której musieliśmy rozwiązać te same zadania w języku R.
Tam również mieliśmy jedną prostą funkcję "sqldf::sqldf()", w której wystarczyło wpisać nasze zapytanie w SQL. Rozwiązania z użyciem pozostałych pakietów przypominały rozwiązanie korzystające ze "zwykłych" funkcji i metod pakietu pandas. Oczywiście w R i Pythonie mamy inną składnię, ale idea roziwązania jest podobna.